In [1]:
from flask import Flask
from flask import Response
from flask import request
from flask_restful import Resource
from flask_restful import Api
from flaskext.mysql import MySQL
from functools import wraps

import json
import re
import sys

sys.path.append("../Common")

from modelChatbot import modelChatbot

#helper functions
def dictToJson(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        res = f(*args, **kwargs)
        res = json.dumps(res, ensure_ascii=False).encode('utf8')
        return Response(res, content_type='application/json; charset=utf-8')
    return decorated_function

#global variables 
app = Flask(__name__)
api = Api(app)    

In [2]:
# 동작하는지를 주소로 확인하기 위해서. 
@app.route("/")
def indexslash():
    return { 'data': u'파워프로그래머' }

@app.route("/index")
def index():
    return { 'data': u'파워프로그래머' }

# db test 
@app.route("/db2json", methods=["GET","POST"])    
def db2json():                    
    config_bible = {
      'user': 'root',
      'password': 'karisma*3%7*4',
      'host': 'mthx.cafe24.com',
      'database': 'bible',
      'raise_on_warnings': True,
    }
    dbbible = dbConMysql( config_bible )
    query = "select * from TB_BIBLE"
    result = dbbible.selectQuery(query)
    print(result)        
    res = json.dumps(result, ensure_ascii=False).encode('utf8')
    return Response(res, content_type='application/json; charset=utf-8')

@app.route("/modtest", methods=["GET","POST"])    
@dictToJson
def modtest():                         
    print( modBot.isExistUsr( 'aaaa' ) )
    modBot.addUser( 'dddd' )            
    modBot.updateUserStat('U9A9agpChomm',0,0,0)        
    print("except")    

@app.route("/keyboard", methods=["GET","POST"])    
def keyboard() :
    print(request)
    return "{\"type\":\"text\"}"

def makeKakaoMsg( szMsg , listMenu ):
    print(szMsg, listMenu )
    if( len(listMenu) != 0 ):
        dicOutput = {"message":{"text":str(szMsg)},"keyboard":{"type":"buttons","buttons":listMenu}}        
    else :
        dicOutput = {"message":{"text":str(szMsg)}}
    return dicOutput

def parseSearch( msg ) :
    pats = ["([가-힣]*)([0-9]*):([0-9]*)(-)([0-9]*)" ,"([가-힣]*)([0-9]*):([0-9]*)(-)" ,"([가-힣]*)([0-9]*):([0-9]*)" ,"([가-힣]*)([0-9]*):" ,"([가-힣]*)([0-9]*)","([가-힣]*)"]
    listDesc = ["book","chap","no_start","isDash","no_end"]    
    dicRet = {}
    for pat in pats : 
        parser = re.compile( pat )
        m = parser.match(msg)
        if( m != None ) : 
            print( m.groups() , type(m.groups()))
            ret_tuple = m.groups()
            if( len(ret_tuple) ) : 
                dicRet['ret']='ok'
                for i in range(0,len(ret_tuple)) :
                    if( ret_tuple[i] != "" ) :                         
                        if( listDesc[i] in ['chap','no_start','no_end'] ) : dicRet[listDesc[i]] = int(ret_tuple[i])
                        else : dicRet[listDesc[i]] = ret_tuple[i]                    
    print( dicRet )            
    return dicRet
            
@app.route("/message", methods=["GET","POST"])    
@dictToJson
def message() :       
    if False : 
        dicJson = json.loads(request.data)
        print(dicJson['user_key'])
        print(dicJson['type'])        
        print(dicJson['content'])
        dicMessage = {"message":{"text":dicJson['content']}}
        return dicMessage
    else :        
        dicJson = json.loads(request.data)        
        usr = dicJson['user_key']
        media_type = dicJson['type']
        msg = dicJson['content']        
        reply,menu = replayFunction( usr, msg )
        ret = makeKakaoMsg(reply,menu)        
        return ret

In [3]:
modBot = modelChatbot()

[dbConMysql] initalized ... = <mysql.connector.connection.MySQLConnection object at 0x7f2cd0096cc0>
[dbConMysql] initalized ... = <mysql.connector.connection.MySQLConnection object at 0x7f2cd00c00f0>


In [4]:

def replayFunction(usr ,msg ):
    listBaseMenu = ['성경검색','성경구절검색','성경요절','성경QT','파파고번역K->E','파파고번역E->K']    
    listMenuMsg = ['기능','메뉴','처음으로','처음']
    
    if( modBot.isExistUsr( usr ) == False ) : 
        modBot.addUser( usr )        
        reply = "새로운 유저이신구요\n %s님 환영합니다\n사용하실 수 있는 기능은 다음과 같아요" % (usr)
        listMenu = listBaseMenu
    else :
        dicUsrStat = modBot.getUserStat(usr)[0];
        print( "dicUsrStat=",dicUsrStat)
                
        # selecting menu 
        if dicUsrStat['statel1'] == 0 : 
            selIdx = 9999 #invalid object
            for i,menu in enumerate(listBaseMenu) :
                if menu == msg : 
                    selIdx = i
                    break
            if( selIdx != 9999 ) : 
                modBot.updateUserStat(usr,selIdx+1,0,0)
                reply = "%s님 %s를 선택하셨군요" % (usr,listBaseMenu[selIdx])
                listMenu = []                
            else: 
                modBot.updateUserStat(usr,0,0,0)
                reply ="%s님 사용하실 수 있는 기능은 다음과 같아요 ^^" % (usr)
                listMenu = listBaseMenu                
        else : 
            if( msg in listMenuMsg ) : 
                modBot.updateUserStat(usr,0,0,0)
                reply ="%s님 기능 메뉴를 다시 보여드릴게요 ^^" % (usr)
                listMenu = listBaseMenu                
            else : 
                # function별 
                if dicUsrStat['statel1'] == 1 : # 성경검색 
                    ret_parse = parseSearch (msg) 
                    ret = modBot.searchBible( ret_parse )
                    if( ret['ret'] == 'ok') :
                        reply ="%s님 선택하신 구절은 다음과 같아요\n%s" % (usr,ret['msg'])                        
                    else :
                        reply ="%s님 선택하신 구절은 유효하지 않습니다." % (usr)
                    listMenu = []                
                elif dicUsrStat['statel1'] == 2:
                    ret = modBot.searchBibleKey( msg )
                    if( ret['ret'] == 'ok') :
                        reply ="%s님 선택하신 키워드 검색 결과는 다음과 같아요\n%s" % (usr,ret['msg'])                        
                    else :
                        reply ="%s님 선택하신 키워드 검색 결과가 없습니다." % (usr)
                    listMenu = []                
                else :                    
                    modBot.updateUserStat(usr,0,0,0)
                    reply ="%s님 기능 메뉴를 다시 보여드릴게요 ^^" % (usr)
                    listMenu = listBaseMenu                                        
    return reply,listMenu


In [ ]:
if __name__ == '__main__':        
    if False :
        app.run(debug=True , host='0.0.0.0')
    else : #jupyer
        from werkzeug.serving import run_simple
        run_simple('0.0.0.0', 5000, app)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
110.76.143.234 - - [15/Oct/2018 14:51:03] "GET /keyboard HTTP/1.1" 200 -


<Request 'http://mthx.cafe24.com:5000/keyboard' [GET]>


[2018-10-15 14:51:06,049] ERROR in app: Exception on /message [POST]
Traceback (most recent call last):
  File "/root/anaconda3/lib/python3.6/site-packages/flask/app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "/root/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/root/anaconda3/lib/python3.6/site-packages/flask_restful/__init__.py", line 273, in error_router
    return original_handler(e)
  File "/root/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/root/anaconda3/lib/python3.6/site-packages/flask/_compat.py", line 35, in reraise
    raise value
  File "/root/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "/root/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1799, in dispatch_request
  

dicUsrStat= {'userkey': 'U9A9agpChomm', 'statel1': 1, 'statel2': 0, 'statel3': 0}
('가나다', '') <class 'tuple'>
('가나다',) <class 'tuple'>
{'ret': 'ok', 'book': '가나다'}
dict_keys(['ret', 'book'])


[2018-10-15 14:51:17,769] ERROR in app: Exception on /message [POST]
Traceback (most recent call last):
  File "/root/anaconda3/lib/python3.6/site-packages/flask/app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "/root/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/root/anaconda3/lib/python3.6/site-packages/flask_restful/__init__.py", line 273, in error_router
    return original_handler(e)
  File "/root/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/root/anaconda3/lib/python3.6/site-packages/flask/_compat.py", line 35, in reraise
    raise value
  File "/root/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "/root/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1799, in dispatch_request
  

...

dicUsrStat= {'userkey': 'U9A9agpChomm', 'statel1': 1, 'statel2': 0, 'statel3': 0}
('요', '1') <class 'tuple'>
('요',) <class 'tuple'>
{'ret': 'ok', 'book': '요', 'chap': 1}
dict_keys(['ret', 'book', 'chap'])
SELECT * from TB_BIBLE_BOOK_DATA WHERE BOOK_SEQNO = 43 and BIBLE_SEQNO in (3,4) and BOOK_CHAP_NO = 1 
U9A9agpChomm님 선택하신 구절은 다음과 같아요
(요1:1) 태초에 말씀이 계시니라 이 말씀이 하나님과 함께 계셨으니 이 말씀은 곧 하나님이시니라 
(요1:2) 그가 태초에 하나님과 함께 계셨고 
(요1:3) 만물이 그로 말미암아 지은 바 되었으니 지은 것이 하나도 그가 없이는 된 것이 없느니라 
(요1:4) 그 안에 생명이 있었으니 이 생명은 사람들의 빛이라 
(요1:5) 빛이 어둠에 비치되 어둠이 깨닫지 못하더라 
(요1:6) 하나님께로부터 보내심을 받은 사람이 있으니 그의 이름은 요한이라 
(요1:7) 그가 증언하러 왔으니 곧 빛에 대하여 증언하고 모든 사람이 자기로 말미암아 믿게 하려 함이라 
(요1:8) 그는 이 빛이 아니요 이 빛에 대하여 증언하러 온 자라 
(요1:9) 참 빛 곧 세상에 와서 각 사람에게 비추는 빛이 있었나니 
(요1:10) 그가 세상에 계셨으며 세상은 그로 말미암아 지은 바 되었으되 세상이 그를 알지 못하였고 
(요1:11) 자기 땅에 오매 자기 백성이 영접하지 아니하였으나 
(요1:12) 영접하는 자 곧 그 이름을 믿는 자들에게는 하나님의 자녀가 되는 권세를 주셨으니 
(요1:13) 이는 혈통으로나 육정으로나 사람의 뜻으로 나지 아니하고 오직 하나님께로부터 난 자들이니라 
(요1:14) 말씀이 육신이 되어 우리 가운데 거하시매 우리가 그의 영광을 보니 아버지의 독생자의 영광이요 은혜

110.76.143.235 - - [15/Oct/2018 14:54:31] "POST /message HTTP/1.1" 200 -


dicUsrStat= {'userkey': 'U9A9agpChomm', 'statel1': 1, 'statel2': 0, 'statel3': 0}
('요', '1', '1', '-', '2') <class 'tuple'>
('요', '1', '1', '-') <class 'tuple'>
('요', '1', '1') <class 'tuple'>
('요', '1') <class 'tuple'>
('요', '1') <class 'tuple'>
('요',) <class 'tuple'>
{'ret': 'ok', 'book': '요', 'chap': 1, 'no_start': 1, 'isDash': '-', 'no_end': 2}
dict_keys(['ret', 'book', 'chap', 'no_start', 'isDash', 'no_end'])
SELECT * from TB_BIBLE_BOOK_DATA WHERE BOOK_SEQNO = 43 and BIBLE_SEQNO in (3,4) and BOOK_CHAP_NO = 1  and BOOK_NO >= 1 and BOOK_NO <= 2 
U9A9agpChomm님 선택하신 구절은 다음과 같아요
(요1:1) 태초에 말씀이 계시니라 이 말씀이 하나님과 함께 계셨으니 이 말씀은 곧 하나님이시니라 
(요1:2) 그가 태초에 하나님과 함께 계셨고 
 []


110.76.143.234 - - [15/Oct/2018 14:57:26] "GET /keyboard HTTP/1.1" 200 -


<Request 'http://mthx.cafe24.com:5000/keyboard' [GET]>
